In [2]:
import openai
import os
from time import time,sleep
import textwrap
import re
import tkinter as tk
from tkinter import filedialog
import ebooklib
from ebooklib import epub
!pip install PyPDF2
import PyPDF2
from bs4 import BeautifulSoup
import codecs

def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()

os.environ["OPENAI_API_KEY"] = open_file('/Path/openai_api_key.txt')
openai.api_key = open_file('/Path/openai_api_key.txt')
openai_api_key = openai.api_key

def save_file(content, filepath):
    with open(filepath, 'w', encoding='utf-8') as outfile:
        outfile.write(content)

# Loop over the EPUB files in the folder
def extract_epub_contents(filepath):
    contents = []
    book = epub.read_epub(filepath)
    chapters = book.get_items_of_type(ebooklib.ITEM_DOCUMENT)
    for chapter in chapters:
        soup = BeautifulSoup(chapter.get_content(), 'html.parser')
        text = soup.get_text()
        contents.append(text)
    return contents

def gpt3_completion(prompt, engine='text-davinci-003', temp=0.6, top_p=1.0, tokens=2000, freq_pen=0.25, pres_pen=0.0, stop=['<<END>>']):
    max_retry = 5
    retry = 0
    prompt = prompt.encode(encoding='ASCII',errors='ignore').decode()
    while True:
        try:
            response = openai.Completion.create(
                engine=engine,
                prompt=prompt,
                temperature=temp,
                max_tokens=tokens,
                top_p=top_p,
                frequency_penalty=freq_pen,
                presence_penalty=pres_pen,
                stop=stop)
            text = response['choices'][0]['text'].strip()
            text = re.sub('\s+', ' ', text)
            filename = '%s_gpt3.txt' % time()
            # with open('gpt3_logs/%s' % filename, 'w') as outfile:
            #    outfile.write('PROMPT:\n\n' + prompt + '\n\n==========\n\nRESPONSE:\n\n' + text)
            return text
        except Exception as oops:
            retry += 1
            if retry >= max_retry:
                return "GPT3 error: %s" % oops
            print('Error communicating with OpenAI:', oops)
            sleep(1)

root = tk.Tk()
root.withdraw()

def filetype(fp):
    ext = os.path.splitext(fp)[-1].lower()

    # Now we can simply use == to check for equality, no need for wildcards.
    if ext == ".pptx":
        print (fp, "is a pptx!")
        PptxReader = download_loader("PptxReader")
        loader = PptxReader()
        documents = loader.load_data(file=Path(fp))
        # Load the PowerPoint presentation
        ppt = pptx.Presentation(file_path)
        # Loop through each slide and extract the text
        for slide in ppt.slides:
            for shape in slide.shapes:
                if shape.has_text_frame:
                    print(shape.text)
    elif ext == ".pdf":
        print (fp, "is a pdf file!")
        with open(fp, 'rb') as f:
            # Create a PdfFileReader object
            reader = PyPDF2.PdfReader(f)
            # Iterate over each page and extract its text
            alltext = ''
            for page in reader.pages:
                alltext += page.extract_text()
    elif ext == ".html":
        with open(fp, 'r') as f:
            reader = BeautifulSoup(f, 'html.parser')
            alltext = reader.get_text()
    elif ext == ".epub":
        print (fp, "is an epub file!")
        alltext = extract_epub_contents(fp)
    else:
        print (fp, "is an unknown file format.")
    print(alltext)
    return alltext

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
fp = filedialog.askopenfilename()

prompt = input("What do you want to summarise, include a '--' in your question?")
if __name__ == '__main__':
    #alltext = ''.join(alltext)
    save_file(prompt + '\n\n<<SUMMARY>>\n', 'prompt_qn.txt')
    chunks = textwrap.wrap(alltext, 2000)
    result = list()
    count = 0
    for chunk in chunks:
        count = count + 1
        prompt = open_file('prompt_qn.txt').replace('<<SUMMARY>>', chunk)
        prompt = prompt.encode(encoding='ASCII',errors='ignore').decode()
        summary = gpt3_completion(prompt)
        print('\n\n\n', count, 'of', len(chunks), ' - ', summary)
        result.append(summary)
    save_file('\n\n'.join(result), 'output_%s.txt' % time())